In [1]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.1.2/libexec")
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import*
from pyspark.sql.types import*
import time

import os

import numpy as np
import re
import pandas as pd 
import traceback
import sys
from time import sleep
import traceback
import smtplib


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
########## Function to Send Email #########

def sendmail(sub, body):
    try:
       SERVER = "us.relay.ibm.com"
       #SERVER = "localhost"
       FROM = "romdbload@radial.com"
       TO = ["mlakshm@us.ibm.com"] # must be a list
       SUBJECT = sub
       TEXT = body
       message = 'From:'+FROM+\
       '\nTo: '+", ".join(TO)+\
       '\nSubject: '+SUBJECT+'\n'+TEXT
       print(message)
       server = smtplib.SMTP(SERVER)
       server.sendmail(FROM, TO, message)
       server.quit()
       return 0
    except Exception as e:
       exp_tb=traceback.format_exc()
       print(exp_tb)


In [4]:
sendmail("Test","Test")

From:romdbload@radial.com
To: mlakshm@us.ibm.com
Subject: Test
Test


0

In [9]:
tbllist=[]
jsonpathlists=[]
colslist=[]

with open('json_column_map.txt') as f:
    contents = f.read()
    contents=contents.strip()
    maplist=contents.split("TABLE_NAME:")
    #print(maplist)
    #maplist = list(filter([], maplist))
    for lmap in maplist:
        lmaplist=lmap.split("\n")
        #lmaplist = list(filter(None, lmaplist))
        tbl=lmaplist.pop(0)
        tbllist.append(tbl)
        collist=[]
        jsonpathlist=[]
        for col_json in lmaplist:
            coljsonarr=col_json.split(",")
            if (len(coljsonarr)==2):
               collist.append(coljsonarr[0])
               jsonpathlist.append(coljsonarr[1])
        colslist.append(collist)
        jsonpathlists.append(jsonpathlist)
            
            
f.close()


tbllist = [x for x in tbllist if x != 'KAFKARADIAL.']
tbllist = [x for x in tbllist if x != '']
colslist = [x for x in colslist if x != []]
jsonpathlists = [x for x in jsonpathlists if x != []]


print(tbllist)
print(jsonpathlists)
print(colslist)

['KAFKARADIAL.ROM_ORDER_HEADER_STG0', 'KAFKARADIAL.ROM_ORDER_LINE_STG0', 'KAFKARADIAL.ROM_RELATED_ORDERS_STG0', 'KAFKARADIAL.ROM_ORDER_PAYMENT_STG0', 'KAFKARADIAL.ROM_ORDER_PROMOTION_STG0', 'KAFKARADIAL.ROM_ORDER_REFERENCES_STG0', 'KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0', 'KAFKARADIAL.ROM_ORDER_LINE_RELATIONSHIP_STG0', 'KAFKARADIAL.ROM_ORDER_CUSTOMER_INFO_STG0', 'KAFKARADIAL.ROM_ORDER_LINE_CHARGES_STG0', 'KAFKARADIAL.ROM_ORDER_LINE_STATUS_STG0']
[['order.id', 'order.sellerId', 'order.customerInfo.phone', 'order.customerInfo.customerId', 'order.customerOrderId', 'order.type', 'order.orderClosed', 'order.orderSourceCode'], ['order.id', 'order.lineItems.0.sizeCode', 'order.sellerId', 'order.lineItems.0.color', 'order.lineItems.0.isAssociateDelivery', 'order.lineItems.0.size', 'order.lineItems.0.colorCode', 'order.lineItems.0.itemDetail.unitOfMeasure', 'order.lineItems.0.itemDetail.itemId', 'order.lineItems.0.itemDetail.gii', 'order.lineItems.0.lineNo'], ['order.id', 'order.sellerId', 'ord

In [6]:

##### Input Tables ######

#tbllist=['KAFKARADIAL.ORDER_HEADER_STG', 'KAFKARADIAL.ORDER_LINE_STG', 'KAFKARADIAL.ORDER_CHARGES_STG']

In [7]:

##### Column Mapping ######

#jsonpathlists=[['msg.order.orderTotal.promotionCharge', 'msg.order.id', 'msg.order.sellerId', 'msg.order.orderTotal.tax', 'msg.order.orderTotal.total', 'msg.order.orderTotal.adjustmentCharge', 'msg.order.type'], ['msg.order.id', 'msg.order.lineItems.0.quantity', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.color', 'msg.order.lineItems.0.lineTotal.totalTaxAmount', 'msg.order.lineItems.0.unitPrice', 'msg.order.lineItems.0.lineTotal.totalPromotionChargeAmount', 'msg.order.lineItems.0.lineTotal.netChargeAmount', 'msg.order.lineItems.0.itemDetail.itemId', 'msg.order.lineItems.0.lineTotal.totalAdjustmentChargeAmount', 'msg.order.lineItems.0.itemDetail.itemDesc', 'lookup'], ['msg.order.id', 'msg.order.lineItems.0.charges.0.id', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.charges.0.detail.amount', 'msg.order.lineItems.0.charges.0.name']]

In [10]:

###### Columns of the Tables #####

#colslist=[['ORDER_PROMO_AMT', 'ORDER_ID', 'SELLER_ID', 'ORDER_TAX_AMT', 'ORDER_TOTAL_AMT', 'ORDER_ADJ_AMT', 'ORDER_TYPE'], ['ORDER_ID', 'ITEM_QTY', 'LINE_NUM', 'COLOR', 'LINE_TAX_AMT', 'ITEM_AMT', 'LINE_PROMO_AMT', 'LINE_TOTAL_AMT', 'ITEM_ID', 'LINE_ADJ_AMT', 'ITEM_DESC', 'COUNTRY_OF_ORIGIN'], ['ORDER_ID', 'CHARGE_ID', 'LINE_NUM', 'CHARGE_AMOUNT', 'CHARGE_NAME']]

In [11]:

########## Reading Input Stream in JSON Format from Kafka ###########

try:

 df = spark \
  .readStream \
  .option("multiLine", True).option("mode", "PERMISSIVE") \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test3") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)
    

In [12]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
###WASHINGTON

jdbcUrl="jdbc:db2://67beb010-4100-48d1-a281-7b5af97791aa.bs2ipa7w0uv9tsomi9ig.databases.appdomain.cloud:31815/bludb:sslConnection=true;"
user="a5406f1c"
password="eK1EhADAU7y3W61T"

In [14]:
###### Lookup Country of Origin #######

key ="'100'"
keyval="SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID="+key

print(keyval)
dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+keyval+') as t').option('user', user).option('password', password).load()
dbl.show()


SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID='100'
+-----------------+
|COUNTRY_OF_ORIGIN|
+-----------------+
+-----------------+



In [15]:
def writeToSQLWarehouse(df,epochId,tbl):
  print(tbl)
  print(df)
  df.write.format("jdbc") \
  .mode("append") \
  .option("url", jdbcUrl) \
  .option("dbtable", tbl) \
  .option("user", user) \
  .option("password", password) \
  .save()

In [16]:
def prepMultiLine(df,epochID):
    df.show()
    l = []
    for row in df.rdd.collect():
       x=row[0]
       l.append(x)
    print(len(l))
    print(l)
    xstr =''.join(l)
    dfr = spark.read.option("multiline", "true") \
      .json(spark.sparkContext.parallelize([xstr]))
    print("***")
    dfr.show()
    print("***")
    return dfr

In [17]:
def prepMultiRows(df,epochID):
    df.show()
    l = []
    for row in df.rdd.collect():
       x=row[0]
       dfr = spark.read.option("multiline", "true") \
       .json(spark.sparkContext.parallelize([x]))
       dfr.show()
       l.append(dfr)
    print("***")

    return l

In [18]:
def popTables(df, epochId):
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    #df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "msg")
    dflist=prepMultiRows(df,epochId)
    print("after")
    for jmsg in dflist:
        jmsg.show()
        #print(jsonpathlist)
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            prepareExpDFR1(jmsg,epochId)
        else:
            print("Corrupt Record:")
            jmsg.show()
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)


In [19]:
###### Data Formatting for JDBC Load #######

def formatVal(val,jsonpathval):

    if val== False or val =='false':
       jsonpathval.append("0")
    elif val is None:
       jsonpathval.append("None")
    elif isinstance(val, (int,np.integer)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    elif isinstance(val, (float,np.float64)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    else:
       jsonpathval.append(val) 


In [20]:
def formatValElm(elm,val,jsonpathval,orderid):
    

    if elm== "orderid" and orderid=="1":
       val = val.replace("-","")
    elif val is None:
       jsonpathval.append("None")
    elif isinstance(val, (int,np.integer)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    elif isinstance(val, (float,np.float64)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    else:
       jsonpathval.append(val) 



In [21]:
lookuptablelist=[["KAFKARADIAL.PRODUCT_MASTER","COUNTRY_OF_ORIGIN"]]

In [22]:
###### Prepare Dataframe and Populate the DB #######

## [[PRODUCT_MASTER,COUNTRY_OF_ORIGIN],[<table2>,<field2>]]

def lookupItemID(lookuptablelist, item_id):
    
    try:
       
     for lookuptable in lookuptablelist:
            
            lookuptblname=lookuptable[0]
            lookuptblfield=lookuptable[1]
            item_id="'"+item_id+"'"
            print(item_id)
            qry='SELECT '+lookuptblfield+' FROM '+lookuptblname+' WHERE ITEM_ID = '+item_id
            print(qry)
            dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+qry+') as t').option('user', user).option('password', password).load()
            dbl.show()
            ctry=dbl.collect()[0][0]
            return ctry

       #### Create Dataframe with JSON value list parsed as the data and the Table Column Names as the Schema ####
       #sdf=spark.createDataFrame(data = jsonvallist, schema = romlistfinal1)
    
    
    except Exception as e:
        print("Exception:")
        print(e)
        exp_tb=traceback.format_exc()
        sendmail("ERROR",exp_tb)
        #print(exp_tb)

In [23]:
#ctry=lookupItemID(lookuptablelist,'100')

In [24]:
import builtins
round = getattr(builtins, "round")

def updateData(jsonvallist,romlistfinal1,tbl):
    
    dict_from_list = dict(zip(romlistfinal1, jsonvallist))
    #print(dict_from_list["ORDER_ID"])
    
    if dict_from_list["ORDER_ID"]=="30002217661":
       amt=round(float(dict_from_list["CHARGE_AMOUNT"]),1)
       dict_from_list["CHARGE_AMOUNT"]=amt
        
    return list(dict_from_list.values())
    #return sdf

In [25]:
jsonvallist=["30002217661","5.017"]
romlistfinal1=["ORDER_ID","CHARGE_AMOUNT"]
jsonvallist=updateData(jsonvallist,romlistfinal1,"TEST")
print(jsonvallist)

['30002217661', 5.0]


In [26]:
import builtins
round = getattr(builtins, "round")
from pyspark.sql.functions import lit

def updateDF(sdf,tbl):
    
    sdf.printSchema()
    if tbl=="KAFKARADIAL.ORDER_CHARGES_STG":
       ordid=sdf.select("ORDER_ID").collect()[0][0]
     
       if ordid=="30002217661":
           chargeamt=sdf.select("CHARGE_AMOUNT").collect()[0][0]
           charge=float(chargeamt)
           charge=round(charge, 1)
           chargeamt1=str(charge)
           sdf=sdf.withColumn("CHARGE_AMOUNT",lit(chargeamt1) )

    
    return sdf
    

In [27]:
###### Prepare Dataframe and Populate the DB #######

def prepareDF(jsonpathval,romlistfinal1,epochId,tbl):
    
    try:
         
       print("&&&&&&&&&&")
       print(jsonpathval)
       ######## Update Data ########
    
       #jsonpathval=updateData(jsonpathval,romlistfinal1,tbl)
       #print(jsonpathval)
    
       jsonval=tuple(jsonpathval)
       jsonvallist=[jsonval]
       print(jsonvallist)
       print(romlistfinal1)
       print("&&&&&&&&&&&&")
    
       
       #### Create Dataframe with JSON value list parsed as the data and the Table Column Names as the Schema ####
       
        
       sdf=createDF(jsonvallist,romlistfinal1)
    
       ######## Update Dataframe ########
        
       #sdf = updateDF(sdf,tbl)

       ##### Explode arrays in the dataframe to form rows for each array element. #####
       for name, dtype in sdf.dtypes:
         try:
           print(name,dtype)
           if dtype.find("array") > -1:
              print("array***************")
              sdf=sdf.withColumn(name, explode(name))
              sdf.show()
         except Exception as e:
            print("dtype exception")
            print(e)
       sdf.show()
       
       writeToSQLWarehouse(sdf, epochId,tbl)
    
    except Exception as e:
        print("hererrerre")
        print(e)

In [28]:
def createDF(jsonvallist,romlistfinal1):
    schema = StructType([])
    sdf = spark.createDataFrame([()], schema)
    try:
           sdf=spark.createDataFrame(data = jsonvallist, schema = romlistfinal1)
    except Exception as e:
           s = "Error {0}".format(str(e))
           if s.find("Unable to infer the type of the field") >-1:
              sarr=s.split(" ")
             
              fieldnm=sarr[-1]
              fieldnmarr=fieldnm.split(".")
              fieldclean=fieldnmarr[0]
              idxrom=romlistfinal1.index(fieldclean)
              print(idxrom)
              print(jsonvallist)
              idxjsonval=jsonvallist[0][idxrom]
              print(idxjsonval)
              tupleX=jsonvallist[0]
              tupleX = [x for x in tupleX if x!=idxjsonval]
              jsonvallist=[tupleX]
              romlistfinal1.remove(fieldclean)
              print(tupleX)
              print(romlistfinal1)
              sdf=createDF(jsonvallist,romlistfinal1)
              createDF(jsonvallist,romlistfinal1)
           else:
            print(e)
            print(jsonvallist)
            print(romlistfinal1)
    sdf.show()
    return sdf

    

In [29]:
def handleArrayVal(df,elmarrstr,elmsplitar):
            
            df.select(elmarrstr+"."+elmsplitar[-1]).show()
            rlist = df.select(elmarrstr+"."+elmsplitar[-1]).collect()
    
            val_list=[]
            val=""
            for r in rlist:
                val_list.append(r[0])
                
            valset=set(val_list)
            if len(valset) == 1:
                val = valset.pop()
            else:
                val = list(valset)
            return val

In [30]:
def explodeArrays(elm,df, elmstr, elmsplitstr):
            
            elmsplitar=elmsplitstr.split(".0.")
            elmsplitarr=elmsplitar[:-1]
            elmstrprep=""
            ctr=0
            elmarrstr=""
            # Example elm: msg.order.lineItems.0.charges.0.id


            for e in elmsplitarr:
                ### examples e: msg.order.lineItems, charges
                esparr=e.split()
                e=esparr[0]
                e=e.replace('"', '')
                
                if ctr==0:
                    elmstrprep=e
                    elmarrstr=elmstrprep
                    
                    elmarrstr=re.sub('[.]','_',elmarrstr)
                    print(e,"**",elmarrstr)
                    ctr=ctr+1
                else:
                   
                   elmarrstr=elmstrprep+"_"+e
                   elmstrprep=elmstrprep+"."+e

                if (elmarrstr not in df.columns):
                   arrchk=df.select(elmstrprep).collect()[0][0]
                   if arrchk:
                      df=df.withColumn(elmarrstr,explode(elmstrprep))
                        
                elmstrprep=elmarrstr
                df.show()
            return df,elmarrstr,elmsplitar


In [31]:
######### Obtaining values from the incoming JSON Msg ###########

def prepareExpDFR1(df,epochId):

#### Iterates for every table that we will be populating ####


 for jsonpathlist,romlistfinal1,tbl in zip(jsonpathlists,colslist,tbllist):
    jsonpathwithval=[]
    jsonpathwithnoval=[]
    romlistwithval=[]
    romlistwithnoval=[]
    jsonpathval=[]
    lookuptablelist=[]
  

    ####### Find values for every column of a table #####

    for elm,colval in zip(jsonpathlist,romlistfinal1):
      elmstr=elm
      elmsplitstr=elm
      print(elm)
      try: 
        
        ########## If the JSON Path contains arrays, explode every array and obtain the values for the JSON Path 
                 
        if (elm.find(".0.")>-1):
            
            df,elmarrstr,elmsplitar=explodeArrays(elm,df, elmstr, elmsplitstr)     
            print("********")
            df.show()
            val=handleArrayVal(df,elmarrstr,elmsplitar)
            jsonpathwithval.append(elm) 
            romlistwithval.append(colval)
            formatVal(val,jsonpathval) 
            
        else:
            
            val = df.select(elm).toPandas().values[0][0]
            print(val)
            jsonpathwithval.append(elm)
            romlistwithval.append(colval)
            formatVal(val,jsonpathval)
     
      

      except Exception as e:
        print(e)
        jsonpathwithnoval.append(elm)
        romlistwithnoval.append(elm)
        #traceback.print_tb
    ########Commenting out the lookup piece ###########
    #lookupcolumn="COUNTRY_OF_ORIGIN"
    #lookupkey="ITEM_ID"
    #print("##################################################")
    #print(jsonpathval)
    #print(romlistwithval)
    #print(romlistfinal1)
    #print(tbl)
    #jsonpathval,romlistwithval=pullCtry(jsonpathval,romlistwithval,romlistfinal1,lookupcolumn,lookupkey,tbl)
    #print("##################################################")
    #print(jsonpathwithval)
    #print(romlistwithval) 
     
    prepareDF(jsonpathval,romlistwithval,epochId,tbl)
    

In [32]:
def pullCtry(jsonpathval,romlistwithval,colslist,lookupcolumn,lookupkey,tbl):
    
    ############# Pull COUNTRY_OF_ORIGIN ##############
    
    try:
    
      if ( lookupcolumn in colslist ):

          print("Inside*******")
        
          item_id_idx=romlistwithval.index(lookupkey)
          item_id=jsonpathval[item_id_idx]
          print(item_id)
          tableinfo=[tbl,lookupcolumn]
          lookuptablelist.append(tableinfo)
       
    
          ctry=lookupItemID(lookuptablelist, item_id)
          jsonpathval.append(ctry)
          romlistwithval.append(lookupcolumn)
     
    except Exception as e:
          print(e)
        
    
    return jsonpathval, romlistwithval
    
    

In [33]:
#pullCtry(["100"],["ITEM_ID"],["COUNTRY_OF_ORIGIN","ITEM_ID"],"COUNTRY_OF_ORIGIN","ITEM_ID","KAFKARADIAL.PRODUCT_MASTER")

In [34]:
####### Writing Steams to DB #######

query1 = df \
    .writeStream \
    .outputMode("append") \
    .option("partition.assignment.strategy", "range")\
    .foreachBatch(popTables)\
    .start()

query1.awaitTermination(10)

<bound method RDD.isEmpty of MapPartitionsRDD[12] at javaToPython at NativeMethodAccessorImpl.java:0>


False

before
+--------------------+
|               value|
+--------------------+
|{  "topicName" : ...|
+--------------------+

+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

***
after
+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+

****
order.id
30002217646
order.sellerId
PROUS
order.customerInfo.phone
1234567890
order.customerInfo.customerId
001231057695
order.customerOrderId
001691000645870
order.type
SALES
order.orderClosed
False
order.orderSourceCode
EB2C
&&&&&&&&&&
['3000221764

order.lineItems.0.itemDetail.itemId
order.lineItems ** order_lineItems
+----------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|
+----------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|
+----------------+--------------------+--------------------+--------------------+

********
+----------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|
+----------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|
+----------------+--------------------+--------------------+--------------------+

+-------------+
|       itemId|
+-------------+
|75-MH06-MD-BK|
+-------------+

order.lineItems.0

+----------------+--------------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|
+----------------+--------------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|
+----------------+--------------------+--------------------+--------------------+--------------------+

********
+----------------+--------------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|
+----------------+--------------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|
+----------------+--------------------+--------------------+---

+-----------------+
|jurisdictionLevel|
+-----------------+
|            STATE|
|           COUNTY|
|            STATE|
|           COUNTY|
+-----------------+

order.id
30002217646
order.lineItems.0.charges.0.taxes.0.situs
order.lineItems ** order_lineItems
+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|order_lineItems_charges|order_lineItems_charges_taxes|
+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {7.8, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Pri

+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|order_lineItems_charges|order_lineItems_charges_taxes|
+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {7.8, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {1.3, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Shipping, []...|         {0.36, Shipping, ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c..

+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_paymentMethods|order_lineItems_charges|order_lineItems_charges_taxes|
+----------------+--------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {7.8, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Price, [], {...|         {1.3, Price, [], ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|{145.5, , CREDIT_...|   {[], Shipping, []...|         {0.36, Shipping, ...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c..

order.id
30002217646
order.sellerId
PROUS
&&&&&&&&&&
['30002217646', 'PROUS']
[('30002217646', 'PROUS')]
['ORDER_ID', 'SELLER_ID']
&&&&&&&&&&&&
+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|    PROUS|
+-----------+---------+

ORDER_ID string
SELLER_ID string
+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|    PROUS|
+-----------+---------+

KAFKARADIAL.ROM_ORDER_LINE_RELATIONSHIP_STG0
DataFrame[ORDER_ID: string, SELLER_ID: string]
order.id
30002217646
order.sellerId
PROUS
&&&&&&&&&&
['30002217646', 'PROUS']
[('30002217646', 'PROUS')]
['ORDER_ID', 'SELLER_ID']
&&&&&&&&&&&&
+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|    PROUS|
+-----------+---------+

ORDER_ID string
SELLER_ID string
+-----------+---------+
|   ORDER_ID|SELLER_ID|
+-----------+---------+
|30002217646|    PROUS|
+-----------+---------+

KAFKARADIAL.ROM_ORDER_CUSTOMER_INFO_STG0
DataFrame[ORDER_ID: string, S

+-------------+
|       itemId|
+-------------+
|75-MH06-MD-BK|
|75-MH06-MD-BK|
|75-MH06-MD-BK|
|75-MH06-MD-BK|
+-------------+

order.type
SALES
&&&&&&&&&&
['1000', '30002217646', 'PROUS', '75-MH06-MD-BK', 'SALES']
[('1000', '30002217646', 'PROUS', '75-MH06-MD-BK', 'SALES')]
['STATUS_ID', 'ORDER_ID', 'SELLER_ID', 'ITEM_ID', 'ORDER_TYPE']
&&&&&&&&&&&&
+---------+-----------+---------+-------------+----------+
|STATUS_ID|   ORDER_ID|SELLER_ID|      ITEM_ID|ORDER_TYPE|
+---------+-----------+---------+-------------+----------+
|     1000|30002217646|    PROUS|75-MH06-MD-BK|     SALES|
+---------+-----------+---------+-------------+----------+

STATUS_ID string
ORDER_ID string
SELLER_ID string
ITEM_ID string
ORDER_TYPE string
+---------+-----------+---------+-------------+----------+
|STATUS_ID|   ORDER_ID|SELLER_ID|      ITEM_ID|ORDER_TYPE|
+---------+-----------+---------+-------------+----------+
|     1000|30002217646|    PROUS|75-MH06-MD-BK|     SALES|
+---------+-----------+--------

In [35]:
query1.stop()

In [36]:
query1.status

{'message': 'Stopped', 'isDataAvailable': False, 'isTriggerActive': False}